In [1]:
## keras_tuner
# The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program.
# Hyperparameters are the variables that govern the training process and the topology of an ML model. 
# Hyperparameters are of two types:
    # Model hyperparameters which influence model selection such as the number and width of hidden layers
    # Algorithm hyperparameters which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

import tensorflow as tf
from tensorflow import keras

import IPython

!pip install -q -U keras-tuner
import kerastuner as kt 

In [3]:
# Download and prepare the dataset
# classifies images of clothing from the Fashion MNIST dataset.

(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0 

In [5]:
# Define the model

def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model 

In [6]:
# Instantiate the tuner and perform hypertuning

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')          

In [8]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True) 

In [9]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



In [10]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4769 - accuracy: 0.8304 - val_loss: 0.4339 - val_accuracy: 0.8466
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3573 - accuracy: 0.8702 - val_loss: 0.3689 - val_accuracy: 0.8679
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3240 - accuracy: 0.8809 - val_loss: 0.3662 - val_accuracy: 0.8686
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2985 - accuracy: 0.8900 - val_loss: 0.3498 - val_accuracy: 0.8762
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2817 - accuracy: 0.8955 - val_loss: 0.3584 - val_accuracy: 0.8724
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2669 - accuracy: 0.9005 - val_loss: 0.3560 - val_accuracy: 0.8736
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2550 - accuracy: 0.9062 - val_loss: 0.3307 - val_accuracy: